In [ ]:
import numpy as np
import torch
import pandas as pd
from tqdm.notebook import tqdm

df = pd.read_csv('training.1600000.processed.noemoticon.csv', 
                 engine='python', 
                 index_col=False,
                 names = ['label', 'id', 'date', 'query', 'user_id', 'tweet'])

df.set_index('id', inplace=True)
df.head()

In [ ]:
df.label.value_counts()

In [ ]:
label_dict = {}
for index, label in enumerate(df.label.unique()):
    label_dict[label] = index

df['label'] = df.label.replace(label_dict)
df.head()

In [ ]:
# shuffle rows and take only 100 000 samples
df = df.sample(frac = 1)[0:100000]
df

# Training/Validation Split

In [ ]:
import numpy as np 
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size=0.15,
    random_state=60
)

In [ ]:
import collections
counter = collections.Counter(y_val)

print(counter)

In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['label', 'data_type']).count()

# Tokenizer & Encoding

In [ ]:
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
# encode text into tokens using bert tokenizer
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].tweet.values, # ... feed training sentences
    add_special_tokens=True, # ... Berts way of knowing sentence start/end
    return_attention_mask=True, # ... using a fixed input (dim will == 256, will tell us where blank info is)
    pad_to_max_length=True, 
    max_length=256,
    return_tensors='pt'
)

# validation
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].tweet.values, # ... feed validation sentences
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

# Split our dataset into input ids, attention masks, and labels
# input ids represent each word as a number
input_ids_train = encoded_data_train['input_ids']
attention_mask_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)
                               
input_ids_val = encoded_data_val['input_ids']
attention_mask_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values) 

In [ ]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_mask_train, 
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_mask_val,
                            labels_val)

# PreTrained BERT Model

In [ ]:
from transformers import BertForSequenceClassification # ... each sequence will fall into a class

In [ ]:
# Fine-tuning step , here we are actually redefining the arcitecture of the model

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', # base version of BERT is alot lighter to train/infer
    num_labels = len(label_dict),
    output_attentions=False, # attending to certain words more than others
    output_hidden_states=False
)

# fetch BERT model (~450MB)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32 # ... particularily small batch size bc of memory limitations


dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train), # prevents the model from learning sequence based differences
    batch_size=batch_size,
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32, # larger batch size in val data, don't have to do many computations--i.e no back propagation
)

# Optimizer and Scheduler

In [ ]:
# defining learning rate and how it changes through each epoch
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
# stochastic optimization approach

optimizer = AdamW(
    model.parameters(),
    lr=1e-5, # 2e-5 > 5e-5
    eps=1e-8
)

In [ ]:
epochs = 10 # can be fine tuned

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs, # num of training steps / learning rate changes 
)

# Defining Performance Metrics

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten()
    
    return f1_score(labels_flat, preds_flat, average='weighted') 
    # f1 is good for a class imbalance, accuracy alone may not be enough
    # weighted vs macro , by use case
    
def accuracy_per_class(preds, labels):
    labels_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = pred_flat[labels_flat==label] # take out all the pred_flat where the true label is the one we care about
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {labels_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])} / {len(y_true)}\n')
        

# Create Training Loop

In [ ]:
import random

seed_val = 6
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val) # only if using GPU

In [ ]:
torch.cuda.is_available()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device) # send model to device

print(device)

In [ ]:
model.to(device)

In [ ]:
# evaluation function

def evaluate(dataloader_val):
    
    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':       batch[0],
                  'attention_mask':  batch[1],
                  'labels':          batch[2]
                 }
        with torch.no_grad():
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
    loss_val_avg = loss_val_total/len(dataloader_val)
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    
    return loss_val_avg, predictions, true_vals

In [ ]:
# epoch loop
for epoch in tqdm(range(1,epochs+1)):
    
    model.train() # ... training mode
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train,
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)
    
    for batch in progress_bar: 
        
        model.zero_grad() # gradients to zero, no need for this when using recurrent networks
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids' : batch[0],
            'attention_mask' : batch[1],
            'labels' : batch[2]
        }
        
        outputs = model(**inputs) # give bert our inputs, unpacks our dict into model
        
        loss = outputs[0] # loss
        loss_train_total += loss.item() 
        loss.backward() # back propoagation step
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # clip gradient (normalize weights to 1)
        
        # step optimizer & scheduler
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))}) # loss of current item / batch length
        # update progress bar and end batch loop
        
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model') # ... save model every epoch
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total/len(dataloader_train)
    
    tqdm.write(f'Training loss: {loss_train_avg}')
    # Use evaluate function to get validation loss
    
    # overtraining occurs when training loss is still going down - but val loss starts to go up
    # validation loss increasing indicates generalizability is lost, overfitting to the train dataset
    
    # evaluate function is similar to training, except it does not carry out backpropagation
    # eval mode freezes all the weights ... ignored/disabled gradients
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

# Load & Evaluate the model

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions=False, 
    output_hidden_states=False
)

In [ ]:
model.to(device)

In [ ]:
model.load_state_dict(
    torch.load('Models/BERT_ft_epoch7.model',
               map_location=torch.device('cuda')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_val)

In [ ]:
accuracy_per_class(predictions, true_vals)